# imports and loading data

In [1]:
# imports
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen

# pandas config
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:,.2f}'.format

# seed reproducibilidade
myseed = 484

In [2]:
# pegando os dados do github e aprindo o json
data_url = "https://github.com/sthemonica/alura-voz/blob/main/Dados/Telco-Customer-Churn.json?raw=true"
response = urlopen(data_url)
data_json = json.loads(response.read())

# carregando o dataframe
df = pd.json_normalize(data_json, max_level=2, sep='_')

# removendo o prefixo
df.columns = ['_'.join(cols.split('_')[1:]) if len(cols.split('_')) > 1 else cols for cols in df.columns]

# mostrando o cabecalho
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4


In [3]:
# classificando colunas em (qualitativos, quantitativos e identificao)
coluna_id = ['customerID']
colunas_quantitativas = ['tenure', 'Charges_Monthly', 'Charges_Total']
colunas_qualitativas = df.drop(colunas_quantitativas + coluna_id, axis=1).columns.tolist()

# dicionario class_var
dict_class_var = { # dicionario classificacao do tipo de variavel
    'QL': 'variavel qualitativa',
    'QT': 'variavel quantitativa',
    'ID': 'variavel de identificacao'
    }

# dicionario informacao das colunas
dict_info_cols = { 
    'customerID': 'número de identificação único de cada cliente',
    'Churn': 'se o cliente deixou ou não a empresa',
    'gender': 'gênero (masculino e feminino)' ,
    'SeniorCitizen': 'informação sobre um cliente ter ou não idade igual ou maior que 65 anos',
    'Partner':  'se o cliente possui ou não um parceiro ou parceira',
    'Dependents': 'se o cliente possui ou não dependentes',
    'tenure':  'meses de contrato do cliente',
    'PhoneService': 'assinatura de serviço telefônico' ,
    'MultipleLines': 'assisnatura de mais de uma linha de telefone' ,
    'InternetService': 'assinatura de um provedor internet' ,
    'OnlineSecurity': 'assinatura adicional de segurança online' ,
    'OnlineBackup': 'assinatura adicional de backup online' ,
    'DeviceProtection': 'assinatura adicional de proteção no dispositivo' ,
    'TechSupport': 'assinatura adicional de suporte técnico, menos tempo de espera',
    'StreamingTV': 'assinatura de TV a cabo',
    'StreamingMovies': 'assinatura de streaming de filmes' ,
    'Contract': 'tipo de contrato',
    'PaperlessBilling': 'se o cliente prefere receber online a fatura',
    'PaymentMethod': 'forma de pagamento',
    'Charges_Monthly': 'total de todos os serviços do cliente por mês',
    'Charges_Total': 'total gasto pelo cliente'
    }

# dicionario traducao coluna
dict_traducao_coluna = {
    'customerID': 'ID', 
    'Churn': 'Churn',
    'gender': 'Genero',
    'SeniorCitizen': 'Senior',
    'Partner': 'Parceiro',
    'Dependents': 'Dependentes',
    'tenure': 'Meses_de_contrato',
    'PhoneService': 'Servico_telefonico',
    'MultipleLines': 'Multiplas_linhas_telefonicas',
    'InternetService': 'Servico_de_internet',
    'OnlineSecurity': 'Seguranca_online',
    'OnlineBackup': 'Backup_online',
    'DeviceProtection': 'Protecao_no_dispositivo',
    'TechSupport': 'Suporte_tecnico',
    'StreamingTV': 'TV_a_cabo',
    'StreamingMovies': 'Streaming_de_filmes',
    'Contract': 'Tipo_de_contrato',
    'PaperlessBilling': 'Fatura_online',
    'PaymentMethod': 'Metodo_de_pagamento',
    'Charges_Monthly': 'Cobrancas_mensais',
    'Charges_Total': 'Cobrancas_total'}

In [4]:
def df_check(df):
    # criando dataframe para verificacao
    check = pd.DataFrame({'name': df.columns})

    # verificando quantos valores unicos existem em cada  (numero maximo possivel 7267)
    check['nunique'] = df.nunique().values

    # verificando quantidades de nulos
    check['isnull'] = df.isnull().sum().values

    # verificando espacos faltantes
    temp_series1 = df[df.select_dtypes('object').columns].apply(lambda x: x.str.strip().isin(['']).sum()) # colunas str
    temp_series2 = df.select_dtypes(['float', 'integer']).isnull().sum() # colunas numericos
    temp_series = pd.concat([temp_series1, temp_series2])
    temp_series.name = 'blank'
    check = check.merge(temp_series, how='left', left_on='name', right_index=True)

    # verificando data type
    check['dtypes'] = df.dtypes.values

    # vericando quais os valores unicos para colunas com 5 ou menos valores unicos
    check['unique'] = df.apply(lambda x: x.unique() if x.nunique() <= 5 else '-').values

    # classifica (qualitativa, quantitativa, identificacao)
    check['class_var'] = check['name'].replace(colunas_qualitativas, 'QL').replace(
        colunas_quantitativas, 'QT').replace(coluna_id, 'ID' )
    
    # informacoes
    info = check['name'].copy()
    for k in dict_info_cols.keys():
        info.mask(info.str.endswith(k), dict_info_cols[k], inplace=True)
    check['info'] = info

    # traducao
    check['traducao'] = check['name'].map(dict_traducao_coluna)

    # mostrar dataframe
    return check
 
check = df_check(df)
check

,name,nunique,isnull,blank,dtypes,unique,class_var,info,traducao
0,customerID,7267,0,0,object,-,ID,número de identificação único de cada cliente,ID
1,Churn,3,0,224,object,"[No, Yes, ]",QL,se o cliente deixou ou não a empresa,Churn
2,gender,2,0,0,object,"[Female, Male]",QL,gênero (masculino e feminino),Genero
3,SeniorCitizen,2,0,0,int64,"[0, 1]",QL,informação sobre um cliente ter ou não idade igual ou maior que 65 anos,Senior
4,Partner,2,0,0,object,"[Yes, No]",QL,se o cliente possui ou não um parceiro ou parceira,Parceiro
5,Dependents,2,0,0,object,"[Yes, No]",QL,se o cliente possui ou não dependentes,Dependentes
6,tenure,73,0,0,int64,-,QT,meses de contrato do cliente,Meses_de_contrato
7,PhoneService,2,0,0,object,"[Yes, No]",QL,assinatura de serviço telefônico,Servico_telefonico
8,MultipleLines,3,0,0,object,"[No, Yes, No phone service]",QL,assisnatura de mais de uma linha de telefone,Multiplas_linhas_telefonicas
9,InternetService,3,0,0,object,"[DSL, Fiber optic, No]",QL,assinatura de um provedor internet,Servico_de_internet


In [5]:
# removendo colunas com Churn igual a ''
df.drop(df[df['Churn'] == ''].index, inplace=True)

# substituir valor faltando numerico por 0
df['Charges_Total'] = pd.to_numeric(df['Charges_Total'], errors='coerce')
df['Charges_Total'].fillna(0, inplace=True)

In [6]:
check = df_check(df)
check

,name,nunique,isnull,blank,dtypes,unique,class_var,info,traducao
0,customerID,7043,0,0,object,-,ID,número de identificação único de cada cliente,ID
1,Churn,2,0,0,object,"[No, Yes]",QL,se o cliente deixou ou não a empresa,Churn
2,gender,2,0,0,object,"[Female, Male]",QL,gênero (masculino e feminino),Genero
3,SeniorCitizen,2,0,0,int64,"[0, 1]",QL,informação sobre um cliente ter ou não idade igual ou maior que 65 anos,Senior
4,Partner,2,0,0,object,"[Yes, No]",QL,se o cliente possui ou não um parceiro ou parceira,Parceiro
5,Dependents,2,0,0,object,"[Yes, No]",QL,se o cliente possui ou não dependentes,Dependentes
6,tenure,73,0,0,int64,-,QT,meses de contrato do cliente,Meses_de_contrato
7,PhoneService,2,0,0,object,"[Yes, No]",QL,assinatura de serviço telefônico,Servico_telefonico
8,MultipleLines,3,0,0,object,"[No, Yes, No phone service]",QL,assisnatura de mais de uma linha de telefone,Multiplas_linhas_telefonicas
9,InternetService,3,0,0,object,"[DSL, Fiber optic, No]",QL,assinatura de um provedor internet,Servico_de_internet


In [7]:
df.sample(10, random_state=myseed)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
3744,5150-ITWWB,No,Male,1,No,No,3,Yes,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.85,335.75
1553,2223-GDSHL,No,Male,0,Yes,Yes,71,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),116.05,"8,297.50"
162,0258-NOKBL,No,Male,0,No,No,3,Yes,No,Fiber optic,No,Yes,Yes,No,No,Yes,Month-to-month,No,Electronic check,90.40,268.45
508,0723-DRCLG,Yes,Female,1,Yes,No,1,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,74.45,74.45
5114,6987-XQSJT,Yes,Female,1,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,79.50,"4,370.25"
2301,3205-MXZRA,No,Male,0,No,No,26,Yes,No,DSL,No,No,No,Yes,Yes,No,One year,No,Credit card (automatic),59.45,"1,507.00"
5725,7821-DPRQE,No,Male,0,Yes,No,68,Yes,Yes,Fiber optic,Yes,Yes,No,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,107.70,"7,320.90"
81,0129-KPTWJ,No,Male,0,Yes,No,72,Yes,No,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,94.65,"6,747.35"
5401,7389-KBFIT,No,Female,0,Yes,Yes,2,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,30.25,63.75
5654,7721-JXEAW,Yes,Male,0,Yes,No,59,No,No phone service,DSL,No,Yes,No,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic),41.05,"2,452.70"


In [18]:
df_qt = df[colunas_quantitativas].copy()
df_qt.head()

,tenure,Charges_Monthly,Charges_Total
0,9,65.60,593.30
1,9,59.90,542.40
2,4,73.90,280.85
3,13,98.00,"1,237.85"
4,3,83.90,267.40


In [9]:
df_qt['calc_total'] =  df_qt.tenure * df_qt.Charges_Monthly
df_qt['diff_total'] =  df_qt.Charges_Total - df_qt.calc_total

In [19]:
df_qt['Charge_Monthly_avg'] = df_qt.Charges_Total / df_qt.tenure
df_qt

,tenure,Charges_Monthly,Charges_Total,Charge_Monthly_avg
0,9,65.60,593.30,65.92
1,9,59.90,542.40,60.27
2,4,73.90,280.85,70.21
3,13,98.00,"1,237.85",95.22
4,3,83.90,267.40,89.13
...,...,...,...,...
7262,13,55.15,742.90,57.15
7263,22,85.10,"1,873.70",85.17
7264,2,50.30,92.75,46.38
7265,67,67.85,"4,627.65",69.07
